<a href="https://colab.research.google.com/github/arlanscort/hidrologia/blob/main/aquisicao_ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Este código realiza coletas e análises hidrológicas a partir de dados abertos do WebService da Agência Nacional de Águas e Saneamento Básico (ANA)

---

In [23]:
import requests
import pandas as pd
import pandas as pd
import xml.etree.ElementTree as ET
import json
!pip install folium
import folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
def coletar_inventario_estacoes():
    url = 'http://telemetriaws1.ana.gov.br/ServiceANA.asmx/HidroInventario'
    params = {
        'nmEstado': '',
        'codEstDE': '',
        'codEstATE': '',
        'tpEst': '',
        'nmEst': '',
        'nmRio': '',
        'codSubBacia': '',
        'codBacia': '',
        'nmMunicipio': '',
        'sgResp': '',
        'sgOper': '',
        'telemetrica': ''
    }
    response = requests.get(url, params=params)
    xml_data = response.content
    root = ET.fromstring(xml_data)
    estacoes = []
    for estacao in root.findall('.//Table'):
        estacao_info = {elem.tag: elem.text for elem in estacao.iter()}
        estacoes.append(estacao_info)
    json_data = json.dumps(estacoes)
    df = pd.read_json(json_data)
    return df

In [5]:
df = coletar_inventario_estacoes()

In [6]:
df.head(3)

,Table,BaciaCodigo,SubBaciaCodigo,RioCodigo,RioNome,EstadoCodigo,nmEstado,MunicipioCodigo,nmMunicipio,ResponsavelCodigo,...,TipoRedeCaptacao,TipoRedeSedimentos,TipoRedeQualAgua,TipoRedeClasseVazao,UltimaAtualizacao,Operando,Descricao,NumImagens,DataIns,DataAlt
0,\n,3,32,NaN,None,5,PARÁ,5063000,SALINÓPOLIS,5,...,1.0,0.0,0.0,0.0,2023-06-19 00:00:00,0,None,0.0,None,2023-06-19 00:00:00
1,\n,3,32,NaN,None,5,PARÁ,5043000,MARACANÃ,56,...,3.0,0.0,0.0,0.0,2005-03-29 00:00:00,1,None,0.0,None,None
2,\n,3,32,NaN,None,5,PARÁ,5063000,SALINÓPOLIS,1,...,7.0,0.0,0.0,0.0,2018-06-18 00:00:00,1,AUTOMÁTICA - IRIS-1,1.0,None,2018-06-18 00:00:00


In [20]:
df.query('Latitude < -26.0 and Latitude > -28 and Longitude > -50.5 and Longitude < -48.5').head()

,Table,BaciaCodigo,SubBaciaCodigo,RioCodigo,RioNome,EstadoCodigo,nmEstado,MunicipioCodigo,nmMunicipio,ResponsavelCodigo,...,TipoRedeCaptacao,TipoRedeSedimentos,TipoRedeQualAgua,TipoRedeClasseVazao,UltimaAtualizacao,Operando,Descricao,NumImagens,DataIns,DataAlt
24854,\n,8,83,NaN,None,23,SANTA CATARINA,23058000,GASPAR,1,...,7.0,0.0,0.0,0.0,2005-08-05 00:00:00,0,None,0.0,None,2010-04-28 00:00:00
24855,\n,8,83,NaN,None,23,SANTA CATARINA,23058000,GASPAR,1,...,7.0,0.0,0.0,0.0,2005-08-05 00:00:00,0,None,0.0,None,2010-04-28 00:00:00
24856,\n,8,83,NaN,None,23,SANTA CATARINA,23058000,GASPAR,1,...,7.0,0.0,0.0,0.0,2005-08-05 00:00:00,0,None,0.0,None,2010-04-28 00:00:00
24857,\n,8,83,NaN,None,23,SANTA CATARINA,23058000,GASPAR,1,...,7.0,0.0,0.0,0.0,2005-08-05 00:00:00,0,None,0.0,None,2010-04-28 00:00:00
24858,\n,8,83,NaN,None,23,SANTA CATARINA,23070000,ILHOTA,1,...,0.0,0.0,0.0,0.0,2018-10-01 00:00:00,0,None,0.0,None,2018-10-01 00:00:00


In [29]:
bnu_lat = -26.9192
bnu_lon = -49.0661
mapa = folium.Map(location=[bnu_lat, bnu_lon], zoom_start=12, width=1200, height=600)
mapa